In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.model_zoo as model_zoo
import torchvision.models as models
from torchvision import transforms, utils
from timm.data.loader import MultiEpochsDataLoader

from model import *
from data_loader import LoadDataset

In [11]:
data_cfgs = {"name": "DL20", "num_classes": 20, "dir":"./data/DL20"}
train_cfgs = {"batch_size": 32, "lr": 0.0001}

### load small version of ResNet
#model = Small_ResNet(BasicBlock, [3, 3, 3], num_classes=data_cfgs['num_classes']).to('cuda')
#model = models.resnext101_32x8d(pretrained=True).to('cuda')

from efficientnet_pytorch import EfficientNet
model = EfficientNet.from_pretrained('efficientnet-b0',num_classes = 20)
#model._fc = nn.Sequential( 
#    nn.Linear(1280, 512, bias=True),
#    nn.Linear(512, 20, bias=True))
#for param in model.parameters():
#    param.requires_grad = True
model = model.to('cuda')

### load train/valid/test dataset
train_dataset = LoadDataset(data_cfgs["dir"], mode="train", random_flip=True)
valid_dataset = LoadDataset(data_cfgs["dir"], mode="valid", random_flip=False)
#test_dataset = LoadDataset(data_cfgs["dir"], mode="test", random_flip=False)

### warp dataset using dataloader
train_dataloader = MultiEpochsDataLoader(train_dataset, batch_size=train_cfgs["batch_size"], shuffle=True, pin_memory=True, drop_last=True, num_workers=64)
valid_dataloader = MultiEpochsDataLoader(valid_dataset, batch_size=train_cfgs["batch_size"], shuffle=False, pin_memory=True, drop_last=False, num_workers=64)
#test_dataloader = MultiEpochsDataLoader(test_dataset, batch_size=1, shuffle=False, pin_memory=True, drop_last=False)

 ### define Adam optimizer: one of the popular optimizers in Deep Learning community
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), train_cfgs["lr"], eps=1e-6)

### define cross-entropy loss for classification
criterion = nn.CrossEntropyLoss()


Loaded pretrained weights for efficientnet-b0


In [3]:
model._fc.in_features

2560

In [4]:
class LabelSmoothing(nn.Module):
    """NLL loss with label smoothing.
    """
    def __init__(self, smoothing=0.0):
        """Constructor for the LabelSmoothing module.
        :param smoothing: label smoothing factor
        """
        super(LabelSmoothing, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing

    def forward(self, x, target):
        logprobs = torch.nn.functional.log_softmax(x, dim=-1)
        nll_loss = -logprobs.gather(dim=-1, index=target.unsqueeze(1))
        nll_loss = nll_loss.squeeze(1)
        smooth_loss = -logprobs.mean(dim=-1)
        loss = self.confidence * nll_loss + self.smoothing * smooth_loss
        return loss.mean()
#criterion = LabelSmoothing(smoothing = 0.3)

In [5]:
#https://gaussian37.github.io/dl-pytorch-lr_scheduler/ 에서 봄
import math
from torch.optim.lr_scheduler import _LRScheduler
from torch.optim.lr_scheduler import ExponentialLR

class CosineAnnealingWarmUpRestarts(_LRScheduler):
    def __init__(self, optimizer, T_0, T_mult=1, eta_max=0.1, T_up=0, gamma=1., last_epoch=-1):
        if T_0 <= 0 or not isinstance(T_0, int):
            raise ValueError("Expected positive integer T_0, but got {}".format(T_0))
        if T_mult < 1 or not isinstance(T_mult, int):
            raise ValueError("Expected integer T_mult >= 1, but got {}".format(T_mult))
        if T_up < 0 or not isinstance(T_up, int):
            raise ValueError("Expected positive integer T_up, but got {}".format(T_up))
        self.T_0 = T_0
        self.T_mult = T_mult
        self.base_eta_max = eta_max
        self.eta_max = eta_max
        self.T_up = T_up
        self.T_i = T_0
        self.gamma = gamma
        self.cycle = 0
        super(CosineAnnealingWarmUpRestarts, self).__init__(optimizer, last_epoch)
        self.T_cur = last_epoch
    
    def get_lr(self):
        if self.T_cur == -1:
            return self.base_lrs
        elif self.T_cur < self.T_up:
            return [(self.eta_max - base_lr)*self.T_cur / self.T_up + base_lr for base_lr in self.base_lrs]
        else:
            return [base_lr + (self.eta_max - base_lr) * (1 + math.cos(math.pi * (self.T_cur-self.T_up) / (self.T_i - self.T_up))) / 2
                    for base_lr in self.base_lrs]

    def step(self, epoch=None):
        if epoch is None:
            epoch = self.last_epoch + 1
            self.T_cur = self.T_cur + 1
            if self.T_cur >= self.T_i:
                self.cycle += 1
                self.T_cur = self.T_cur - self.T_i
                self.T_i = (self.T_i - self.T_up) * self.T_mult + self.T_up
        else:
            if epoch >= self.T_0:
                if self.T_mult == 1:
                    self.T_cur = epoch % self.T_0
                    self.cycle = epoch // self.T_0
                else:
                    n = int(math.log((epoch / self.T_0 * (self.T_mult - 1) + 1), self.T_mult))
                    self.cycle = n
                    self.T_cur = epoch - self.T_0 * (self.T_mult ** n - 1) / (self.T_mult - 1)
                    self.T_i = self.T_0 * self.T_mult ** (n)
            else:
                self.T_i = self.T_0
                self.T_cur = epoch
                
        self.eta_max = self.base_eta_max * (self.gamma**self.cycle)
        self.last_epoch = math.floor(epoch)
        for param_group, lr in zip(self.optimizer.param_groups, self.get_lr()):
            param_group['lr'] = lr
            
#scheduler = CosineAnnealingWarmUpRestarts(optimizer, T_0=150, T_mult=1, eta_max=0.1,  T_up=10, gamma=0.5)
scheduler = ExponentialLR(optimizer, gamma=0.95)

In [6]:
def train_model(model, patience, num_epochs):

    train_losses = []
    valid_losses = []
    mean_train_losses = []
    mean_valid_losses = [] 
    p = 0
    min_valid_loss = 100

    for epoch in range(1, num_epochs + 1):

        model.train() 
        for train_data, target in train_dataloader:
            if torch.cuda.is_available():
                device = torch.device("cuda")
                train_data, target = train_data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(train_data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())
            #scheduler.step() #내가 추가한거

        model.eval() 
        for valid_data, target in valid_dataloader:
            if torch.cuda.is_available():
                device = torch.device("cuda")
                valid_data, target = valid_data.to(device), target.to(device)
            output = model(valid_data)
            loss = criterion(output, target)
            valid_losses.append(loss.item())

        train_loss = np.mean(train_losses)
        valid_loss = np.mean(valid_losses)
        mean_train_losses.append(train_loss)
        mean_valid_losses.append(valid_loss)
        
        if min_valid_loss > valid_loss:
            min_valid_loss = valid_loss
            # print(f'min_valid_loss: {min_valid_loss}')

        epoch_len = len(str(num_epochs))
        print_msg = (f'[{epoch:>{epoch_len}}/{num_epochs:>{epoch_len}}] ' +
                     f'train_loss: {train_loss:.5f} ' +
                     f'valid_loss: {valid_loss:.5f}')
        print(print_msg)
        
        train_losses = []
        valid_losses = []
        
        print_test_accuracy(model, valid_dataloader) # 여따 넣어도 되나
        
        if min_valid_loss < valid_loss and epoch > 1:
            p = p + 1
            #print(f'patience: {p}')
        else:
            p = 0
            torch.save(model.state_dict(), 'bestmodel.pt')
            print("Saving Model...")
        
        if patience == p:
            print("Early stopping")
            break

    model.load_state_dict(torch.load('bestmodel.pt'))

    return  model, mean_train_losses, mean_valid_losses

In [55]:
def test_csv_file(model):
    print("Saving Test.csv… …")

    model.load_state_dict(torch.load('bestmodel.pt'))
    model.eval()
    arr = []
    with torch.no_grad():
        for images, file_names in iter(test_dataloader):
            each_arr = []
            images = images.to('cuda')
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            each_arr.append(file_names[0])
            each_arr.append(int(predicted))
            arr.append(each_arr)

    arr = np.array(arr)
    df = pd.DataFrame(arr, columns=['Id', 'Category'])
    df.to_csv('test.csv', index=False)

test_csv_file(final_model)

Saving Test.csv… …


NameError: name 'test_dataloader' is not defined

In [10]:
def weight_reset(m):
    if isinstance(m, nn.Linear):
        m.reset_parameters()

def print_test_accuracy(model, test_loader):
  class_correct = list(0. for i in range(20))
  class_total = list(0. for i in range(20))

  model.eval()

  with torch.no_grad():
    for test_data, target in test_loader:
        if torch.cuda.is_available():
            device = torch.device("cuda")
            test_data, target = test_data.to(device), target.to(device)
        output = model(test_data)
        _, predicted = torch.max(output, 1)
        c = np.squeeze(predicted.eq(target.data.view_as(predicted)))
        for i in range(len(target.data)):
            label = target.data[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

  model.apply(weight_reset)

  print('Test Accuracy (Total): %.2f %% (%d/%d)\n' % (
      100. * np.sum(class_correct) / np.sum(class_total),
      np.sum(class_correct), np.sum(class_total)))

b0 , smooth labeling

In [9]:
num_epochs = 1000
patience = 15

final_model, train_losses, valid_losses= train_model(model, patience, num_epochs)
print_test_accuracy(final_model, valid_dataloader)

[   1/1000] train_loss: 1.83452 valid_loss: 1.72721
Test Accuracy (Total): 86.86 % (1448/1667)

Saving Model...
[   2/1000] train_loss: 1.97010 valid_loss: 1.69423
Test Accuracy (Total): 87.82 % (1464/1667)

Saving Model...
[   3/1000] train_loss: 1.88279 valid_loss: 1.68182
Test Accuracy (Total): 88.12 % (1469/1667)

Saving Model...
[   4/1000] train_loss: 1.85174 valid_loss: 1.67763
Test Accuracy (Total): 88.18 % (1470/1667)

Saving Model...
[   5/1000] train_loss: 1.79017 valid_loss: 1.66954
Test Accuracy (Total): 88.66 % (1478/1667)

Saving Model...
[   6/1000] train_loss: 1.76670 valid_loss: 1.66932
Test Accuracy (Total): 88.84 % (1481/1667)

Saving Model...
[   7/1000] train_loss: 1.74265 valid_loss: 1.66351
Test Accuracy (Total): 89.20 % (1487/1667)

Saving Model...
[   8/1000] train_loss: 1.73055 valid_loss: 1.67022
Test Accuracy (Total): 89.08 % (1485/1667)

[   9/1000] train_loss: 1.71022 valid_loss: 1.66880
Test Accuracy (Total): 88.72 % (1479/1667)

[  10/1000] train_loss: 

b0 resize 224

In [12]:
num_epochs = 1000
patience = 15

final_model, train_losses, valid_losses= train_model(model, patience, num_epochs)
print_test_accuracy(final_model, valid_dataloader)

[   1/1000] train_loss: 2.04568 valid_loss: 0.99604
Test Accuracy (Total): 75.94 % (1266/1667)

Saving Model...
[   2/1000] train_loss: 1.32789 valid_loss: 0.61501
Test Accuracy (Total): 84.22 % (1404/1667)

Saving Model...
[   3/1000] train_loss: 1.08011 valid_loss: 0.52910
Test Accuracy (Total): 85.84 % (1431/1667)

Saving Model...
[   4/1000] train_loss: 0.91598 valid_loss: 0.49725
Test Accuracy (Total): 86.68 % (1445/1667)

Saving Model...
[   5/1000] train_loss: 0.80801 valid_loss: 0.45589
Test Accuracy (Total): 86.14 % (1436/1667)

Saving Model...
[   6/1000] train_loss: 0.72587 valid_loss: 0.45264
Test Accuracy (Total): 87.10 % (1452/1667)

Saving Model...
[   7/1000] train_loss: 0.65294 valid_loss: 0.41709
Test Accuracy (Total): 87.82 % (1464/1667)

Saving Model...
[   8/1000] train_loss: 0.60336 valid_loss: 0.43309
Test Accuracy (Total): 88.48 % (1475/1667)

[   9/1000] train_loss: 0.55257 valid_loss: 0.44006
Test Accuracy (Total): 87.82 % (1464/1667)



KeyboardInterrupt: 

In [13]:
num_epochs = 1000
patience = 15

final_model, train_losses, valid_losses= train_model(model, patience, num_epochs)
print_test_accuracy(final_model, valid_dataloader)

RuntimeError: DataLoader worker (pid(s) 26167, 26168, 26169, 26170, 26171, 26172, 26173, 26174, 26175, 26176, 26177, 26178, 26179, 26180, 26181, 26182, 26183, 26184, 26185, 26186, 26187, 26188, 26189, 26190, 26191, 26192, 26193, 26194, 26195, 26196, 26197, 26198, 26199, 26200, 26201, 26202, 26203, 26204, 26205, 26206, 26207, 26208, 26209, 26210, 26211, 26212, 26213, 26214, 26215, 26216, 26217, 26218, 26219, 26220, 26221, 26222, 26223, 26224, 26225, 26226, 26227, 26228, 26229, 26230) exited unexpectedly

resize(600)
b7/finetuning 없음 (batch size 2)

In [8]:
num_epochs = 1000
patience = 5

final_model, train_losses, valid_losses= train_model(model, patience, num_epochs)
print_test_accuracy(final_model, valid_dataloader)

[   1/1000] train_loss: 4.09283 valid_loss: 3.85271
Saving Model...
[   2/1000] train_loss: 3.50324 valid_loss: 4.67777
[   3/1000] train_loss: 3.26275 valid_loss: 4.68264
[   4/1000] train_loss: 3.12964 valid_loss: 5.53166
[   5/1000] train_loss: 3.01338 valid_loss: 5.95381
[   6/1000] train_loss: 2.95209 valid_loss: 6.44378
Early stopping


IndexError: invalid index of a 0-dim tensor. Use `tensor.item()` in Python or `tensor.item<T>()` in C++ to convert a 0-dim tensor to a number

resize(256) 추가

In [10]:
num_epochs = 1000
patience = 15

final_model, train_losses, valid_losses= train_model(model, patience, num_epochs)
print_test_accuracy(final_model, valid_dataloader)

[   1/1000] train_loss: 3.68471 valid_loss: 3.19933
Saving Model...
[   2/1000] train_loss: 3.05829 valid_loss: 3.11481
Saving Model...
[   3/1000] train_loss: 2.90352 valid_loss: 3.11834
[   4/1000] train_loss: 2.84723 valid_loss: 3.04560
Saving Model...
[   5/1000] train_loss: 2.80475 valid_loss: 3.06946
[   6/1000] train_loss: 2.78334 valid_loss: 3.04708
[   7/1000] train_loss: 2.76857 valid_loss: 3.08668
[   8/1000] train_loss: 2.77765 valid_loss: 3.10754
[   9/1000] train_loss: 2.76548 valid_loss: 3.09385
[  10/1000] train_loss: 2.76035 valid_loss: 3.12681
[  11/1000] train_loss: 2.76435 valid_loss: 3.08634
[  12/1000] train_loss: 2.76891 valid_loss: 3.09001
[  13/1000] train_loss: 2.73274 valid_loss: 3.08450
[  14/1000] train_loss: 2.73949 valid_loss: 3.15281
[  15/1000] train_loss: 2.80030 valid_loss: 3.24563
[  16/1000] train_loss: 2.75570 valid_loss: 3.11533
[  17/1000] train_loss: 2.72768 valid_loss: 3.13789
[  18/1000] train_loss: 2.71999 valid_loss: 3.08311
[  19/1000] trai

models.resnext101_32x8d(pretrained=True, progress = True).to('cuda')  쓴거
rl = 0.001
batchsize = 200

In [38]:
num_epochs = 1000
patience = 15

final_model, train_losses, valid_losses= train_model(model, patience, num_epochs)
print_test_accuracy(final_model, valid_dataloader)

[   1/1000] train_loss: 2.49489 valid_loss: 3.02104
Saving Model...
[   2/1000] train_loss: 1.22229 valid_loss: 1.57715
Saving Model...
[   3/1000] train_loss: 0.87225 valid_loss: 1.44397
Saving Model...
[   4/1000] train_loss: 0.63376 valid_loss: 1.52186
[   5/1000] train_loss: 0.50805 valid_loss: 1.49858
[   6/1000] train_loss: 0.44530 valid_loss: 2.06596
[   7/1000] train_loss: 0.40726 valid_loss: 1.71300
[   8/1000] train_loss: 0.26045 valid_loss: 1.47815
[   9/1000] train_loss: 0.20722 valid_loss: 2.00285
[  10/1000] train_loss: 0.22510 valid_loss: 1.99222
[  11/1000] train_loss: 0.36936 valid_loss: 1.76069
[  12/1000] train_loss: 0.89980 valid_loss: 1.68771
[  13/1000] train_loss: 0.41474 valid_loss: 1.48265
[  14/1000] train_loss: 0.18906 valid_loss: 2.00617
[  15/1000] train_loss: 0.18387 valid_loss: 1.58351
[  16/1000] train_loss: 0.12665 valid_loss: 1.72464
[  17/1000] train_loss: 0.09174 valid_loss: 1.64384
[  18/1000] train_loss: 0.07164 valid_loss: 1.74159
Early stopping
T

models.resnext101_32x8d(pretrained=True, progress = True).to('cuda')  쓴거
label smoothing = 0.3 해본거 뭔지 모르겠음.  (https://stackoverflow.com/questions/55681502/label-smoothing-in-pytorch)
nvidat 뭐시기 씀

In [44]:
num_epochs = 1000
patience = 15

final_model, train_losses, valid_losses= train_model(model, patience, num_epochs)
print_test_accuracy(final_model, valid_dataloader)

[   1/1000] train_loss: 4.98006 valid_loss: 3.78770
Saving Model...
[   2/1000] train_loss: 3.42048 valid_loss: 3.47698
Saving Model...
[   3/1000] train_loss: 3.06916 valid_loss: 3.40553
Saving Model...
[   4/1000] train_loss: 2.92167 valid_loss: 3.36669
Saving Model...
[   5/1000] train_loss: 2.83931 valid_loss: 3.33530
Saving Model...
[   6/1000] train_loss: 2.79113 valid_loss: 3.32254
Saving Model...
[   7/1000] train_loss: 2.76962 valid_loss: 3.32636
[   8/1000] train_loss: 2.74539 valid_loss: 3.31648
Saving Model...
[   9/1000] train_loss: 2.72877 valid_loss: 3.34809
[  10/1000] train_loss: 2.72195 valid_loss: 3.31191
Saving Model...
[  11/1000] train_loss: 2.71174 valid_loss: 3.31319
[  12/1000] train_loss: 2.70807 valid_loss: 3.31754
[  13/1000] train_loss: 2.70332 valid_loss: 3.33277
[  14/1000] train_loss: 2.69845 valid_loss: 3.32785
[  15/1000] train_loss: 2.69757 valid_loss: 3.32530
[  16/1000] train_loss: 2.69613 valid_loss: 3.33687
[  17/1000] train_loss: 2.69852 valid_lo

models.resnext101_32x8d(pretrained=True, progress = True).to('cuda')  쓴거
label smoothing = 0.3 해본거 뭔지 모르겠음.  (https://stackoverflow.com/questions/55681502/label-smoothing-in-pytorch)
nvidat 뭐시기 씀  
+learning rate schedule : exponetial 써봄

In [53]:
num_epochs = 1000
patience = 15

final_model, train_losses, valid_losses= train_model(model, patience, num_epochs)
print_test_accuracy(final_model, valid_dataloader)

[   1/1000] train_loss: 5.54712 valid_loss: 4.67650
Saving Model...
[   2/1000] train_loss: 4.30602 valid_loss: 4.47192
Saving Model...
[   3/1000] train_loss: 4.24488 valid_loss: 4.46818
Saving Model...
[   4/1000] train_loss: 4.23627 valid_loss: 4.45199
Saving Model...
[   5/1000] train_loss: 4.24401 valid_loss: 4.42681
Saving Model...
[   6/1000] train_loss: 4.24099 valid_loss: 4.44410
[   7/1000] train_loss: 4.24283 valid_loss: 4.47978
[   8/1000] train_loss: 4.24101 valid_loss: 4.46950
[   9/1000] train_loss: 4.23962 valid_loss: 4.43817
[  10/1000] train_loss: 4.23426 valid_loss: 4.45474
[  11/1000] train_loss: 4.24901 valid_loss: 4.45258
[  12/1000] train_loss: 4.24028 valid_loss: 4.44296
[  13/1000] train_loss: 4.24799 valid_loss: 4.46802
[  14/1000] train_loss: 4.24406 valid_loss: 4.47880
[  15/1000] train_loss: 4.24200 valid_loss: 4.42268
Saving Model...
[  16/1000] train_loss: 4.23380 valid_loss: 4.41957
Saving Model...
[  17/1000] train_loss: 4.23507 valid_loss: 4.44467
[  1

models.resnext101_32x8d(pretrained=True, progress = True).to('cuda')  쓴거

In [30]:
num_epochs = 1000
patience = 15

final_model, train_losses, valid_losses= train_model(model, patience, num_epochs)
print_test_accuracy(final_model, valid_dataloader)

[   1/1000] train_loss: 3.34607 valid_loss: 1.29235
Saving Model...
[   2/1000] train_loss: 0.79014 valid_loss: 0.97661
Saving Model...
[   3/1000] train_loss: 0.30339 valid_loss: 0.99960
[   4/1000] train_loss: 0.15766 valid_loss: 1.06899
[   5/1000] train_loss: 0.08559 valid_loss: 1.18780
[   6/1000] train_loss: 0.08409 valid_loss: 1.13143
[   7/1000] train_loss: 0.05148 valid_loss: 1.23776
[   8/1000] train_loss: 0.04816 valid_loss: 1.19748
[   9/1000] train_loss: 0.04579 valid_loss: 1.23585
[  10/1000] train_loss: 0.03984 valid_loss: 1.32761
[  11/1000] train_loss: 0.05532 valid_loss: 1.31975
[  12/1000] train_loss: 0.05877 valid_loss: 1.26559
[  13/1000] train_loss: 0.05935 valid_loss: 1.26448
[  14/1000] train_loss: 0.06302 valid_loss: 1.26687
[  15/1000] train_loss: 0.04225 valid_loss: 1.23406
[  16/1000] train_loss: 0.03199 valid_loss: 1.36592
[  17/1000] train_loss: 0.03877 valid_loss: 1.40870
Early stopping
Test Accuracy (Total): 71.21 % (1187/1667)



models.resnext101_32x8d(pretrained=False, progress = True).to('cuda')  쓴거

In [17]:
num_epochs = 1000
patience = 15

final_model, train_losses, valid_losses= train_model(model, patience, num_epochs)
print_test_accuracy(final_model, valid_dataloader)

[   1/1000] train_loss: 3.63468 valid_loss: 3.24151
Saving Model...
[   2/1000] train_loss: 2.99377 valid_loss: 2.98866
Saving Model...
[   3/1000] train_loss: 2.95420 valid_loss: 2.93517
Saving Model...
[   4/1000] train_loss: 2.89297 valid_loss: 2.89309
Saving Model...
[   5/1000] train_loss: 2.82801 valid_loss: 2.81104
Saving Model...
[   6/1000] train_loss: 2.77810 valid_loss: 2.75461
Saving Model...
[   7/1000] train_loss: 2.71515 valid_loss: 2.74266
Saving Model...
[   8/1000] train_loss: 2.67226 valid_loss: 2.75661
[   9/1000] train_loss: 2.60114 valid_loss: 2.65724
Saving Model...
[  10/1000] train_loss: 2.54870 valid_loss: 2.69271
[  11/1000] train_loss: 2.49091 valid_loss: 2.64498
Saving Model...
[  12/1000] train_loss: 2.41394 valid_loss: 2.65417
[  13/1000] train_loss: 2.36246 valid_loss: 2.70276
[  14/1000] train_loss: 2.34232 valid_loss: 2.68470
[  15/1000] train_loss: 2.23078 valid_loss: 2.65275
[  16/1000] train_loss: 2.13045 valid_loss: 2.65005
[  17/1000] train_loss: 

models.wide_resnet50_2(pretrained=False).to('cuda') 쓴거

In [21]:
num_epochs = 1000
patience = 15

final_model, train_losses, valid_losses= train_model(model, patience, num_epochs)
print_test_accuracy(final_model, valid_dataloader)

[   1/1000] train_loss: 3.57445 valid_loss: 2.95327
Saving Model...
[   2/1000] train_loss: 2.83149 valid_loss: 2.75759
Saving Model...
[   3/1000] train_loss: 2.65120 valid_loss: 2.66315
Saving Model...
[   4/1000] train_loss: 2.51122 valid_loss: 2.56807
Saving Model...
[   5/1000] train_loss: 2.35330 valid_loss: 2.57100
[   6/1000] train_loss: 2.24620 valid_loss: 2.48517
Saving Model...
[   7/1000] train_loss: 2.14792 valid_loss: 2.47959
Saving Model...
[   8/1000] train_loss: 1.97883 valid_loss: 2.61383
[   9/1000] train_loss: 1.82066 valid_loss: 2.62226
[  10/1000] train_loss: 1.61720 valid_loss: 2.65290
[  11/1000] train_loss: 1.46929 valid_loss: 2.57710
[  12/1000] train_loss: 1.28618 valid_loss: 2.87399
[  13/1000] train_loss: 1.13475 valid_loss: 2.91118
[  14/1000] train_loss: 0.94653 valid_loss: 3.01158
[  15/1000] train_loss: 0.83789 valid_loss: 3.18958
[  16/1000] train_loss: 0.70916 valid_loss: 3.17891
[  17/1000] train_loss: 0.58533 valid_loss: 3.20930
[  18/1000] train_lo

models.wide_resnet50_2(pretrained=True).to('cuda') 쓴거

In [26]:
num_epochs = 1000
patience = 15

final_model, train_losses, valid_losses= train_model(model, patience, num_epochs)
print_test_accuracy(final_model, valid_dataloader)

[   1/1000] train_loss: 6.23711 valid_loss: 2.49377
Saving Model...
[   2/1000] train_loss: 1.34758 valid_loss: 1.59409
Saving Model...
[   3/1000] train_loss: 0.63932 valid_loss: 1.44614
Saving Model...
[   4/1000] train_loss: 0.29878 valid_loss: 1.42573
Saving Model...
[   5/1000] train_loss: 0.16961 valid_loss: 1.38099
Saving Model...
[   6/1000] train_loss: 0.09423 valid_loss: 1.40760
[   7/1000] train_loss: 0.05140 valid_loss: 1.43718
[   8/1000] train_loss: 0.03289 valid_loss: 1.50548
[   9/1000] train_loss: 0.02314 valid_loss: 1.46350
[  10/1000] train_loss: 0.01942 valid_loss: 1.44684
[  11/1000] train_loss: 0.01234 valid_loss: 1.47245
[  12/1000] train_loss: 0.00733 valid_loss: 1.48660
[  13/1000] train_loss: 0.00534 valid_loss: 1.51482
[  14/1000] train_loss: 0.00493 valid_loss: 1.51713
[  15/1000] train_loss: 0.00627 valid_loss: 1.54443
[  16/1000] train_loss: 0.00427 valid_loss: 1.58538
[  17/1000] train_loss: 0.00418 valid_loss: 1.57446
[  18/1000] train_loss: 0.00300 vali